In [1]:
# instalar conda install requests lxml autopep8 openpyxl

In [33]:
MAIN_URL = 'https://www.computrabajo.com.co/ofertas-de-trabajo/?by=publicationtime&pubdate=1&sal=5'
SLICE_URL = 'https://www.computrabajo.com.co'
XPATH_link = '//div[@class="gO"]/div//h2/a/@href'
XPATH_name = '//div[@class="gO"]/div//h2/a/text()'
XPATH_city = '//div[@class="gO"]/div//div//span[@itemprop="addressLocality"]/a/text()'
XPATH_city2 = '//div[@class="gO"]/div//div//span[@itemprop="addressRegion"]/a/text()'
XPATH_summary = '//div[@class="gO"]/div//p/text()'
XPATH_company = '//div[@class="gO"]/div//div//span[@itemprop="name"]'
XPATH_company2 = '//div[@class="gO"]/div//div//span[@itemprop="name"]/a/text()'
XPATH_next_pag = '//div[@class="paginas paginasCenter"]//li[@title="Siguiente"]/a/@href'
MAIN = 'Computrabajo'

In [83]:
import requests
import lxml.html as html
import pandas as pd
from datetime import datetime

def extract_data(parsed):
    link = parsed.xpath(XPATH_link)
    link = [SLICE_URL + i for i in link]
    name = parsed.xpath(XPATH_name)
    city = parsed.xpath(XPATH_city)
    city2 = parsed.xpath(XPATH_city2)
    summary = parsed.xpath(XPATH_summary)
    company = company_extract(parsed.xpath(XPATH_company), parsed.xpath(XPATH_company2))
    date = [f'{datetime.today().day}-{datetime.today().month}-{datetime.today().year}' for _ in range(len(link))]
    source = [MAIN for _ in range(len(link))]
    vacantes = []
    for i in range(len(link)):
        vacantes.append([name[i], date[i], company[i], link[i], summary[i], source[i], city[i], city2[i]])
    return vacantes

def company_extract(first, second):
    companies = []
    j = 0
    for i in first:
        company = i.xpath('text()')[0].replace('\r','').replace('\n','').strip()
        
        if company:
            companies.append(company.strip())
        else:
            companies.append(second[j])
            j += 1
    return companies


def parse_home(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            home = response.content.decode('utf-8')
            parsed = html.fromstring(home)
            data = extract_data(parsed)
            next_link = parsed.xpath(XPATH_next_pag)
            print(next_link)
        else:
            raise ValueError(f'Error: {response.status_code}')
    except ValueError as ve:
        print(ve)
        
    return data, next_link

def run():
    data = []
    link = MAIN_URL
    while len(link) != 0:
        transitoria, link = parse_home(link)
        data.extend(transitoria)
        if len(link) != 0:
            link = link[0]
    data = pd.DataFrame(data, columns=['Vacante', 'Fecha', 'Compania', 'Link', 'Informacion_Adicional',
                                       'Fuente', 'Departamento', 'Ciudad'])
    
    data.to_excel(f'data_{data.iloc[0, 1]}.xlsx', index=False)
    return data

if __name__ == '__main__':
    data = run()
    print(data)

['https://www.computrabajo.com.co/ofertas-de-trabajo/?pubdate=1&sal=5&p=23']
['https://www.computrabajo.com.co/ofertas-de-trabajo/?pubdate=1&sal=5&p=24']
['https://www.computrabajo.com.co/ofertas-de-trabajo/?pubdate=1&sal=5&p=25']
[]
                                              Vacante     Fecha  \
0                       Analista de logistica Guacarí  3-2-2021   
1   Ingeniero/a Mecánico Especialista en salud ocu...  3-2-2021   
2                          Coordinador sostenibilidad  3-2-2021   
3   Grupo Aval / Asesor Comercial o Ejecutivo externo  3-2-2021   
4                          Coordinador de operaciones  3-2-2021   
..                                                ...       ...   
67                   Jefe Comercial Regional Medellín  3-2-2021   
68                  Especialista en Marketing Digital  3-2-2021   
69                  Enfermero Auditor Cuentas Medicas  3-2-2021   
70                             Medico Hospitalización  3-2-2021   
71                           